In [73]:
from selenium import webdriver
import time
import urllib.request 
from PIL import Image
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
from html_table_extractor.extractor import Extractor
from bs4 import BeautifulSoup

In [74]:
def table_to_markdown(table):
    """
    Chuyển đổi phần tử bảng HTML thành văn bản Markdown.

    Parameters:
        table (bs4.element.Tag): Phần tử bảng HTML.

    Returns:
        str: Văn bản Markdown của bảng.
    """
    markdown_table = ""
    # Số cột của bảng
    num_columns = len(table.find_all('th'))

    # Header của bảng
    markdown_table += "| " + " | ".join([th.get_text().strip() for th in table.find_all('th')]) + " |\n"
    markdown_table += "|-" + "-|-".join(["" for _ in range(num_columns)]) + "|\n"

    # Dữ liệu của bảng
    for row in table.find_all('tr')[1:]:
        markdown_table += "| " + " | ".join([td.get_text().strip() for td in row.find_all('td')]) + " |\n"
    return markdown_table

In [151]:
def link_retriever(n, file_path):
    """
    A function for retriever a .txt of links based on provided number

    Input: n (the number of pages - each page has 10 link)
    Output: a list of link
            .txt file
    """
    driver = webdriver.Chrome()
    driver.get('https://dichvucong.gov.vn/p/home/dvc-dich-vu-cong-truc-tuyen-ds.html?pDoiTuongId=1')

    desired_page_number = n
    page_number = "//div[@class='col-sm-8 col-xs-12']/div[4]//li[@jp-data = '{}']"

    link_dvc = []
    dvc_container = driver.find_elements("xpath","//div[@class='col-sm-8 col-xs-12']/div[3]//li//a")
    for link_container in dvc_container:
        link = link_container.get_attribute("href")
        link_dvc.append(link)
    
    for i in range(2, 11):
        container = driver.find_element("xpath",page_number.format(i)).click()
        time.sleep(15)
    
        dvc_container = driver.find_elements("xpath","//div[@class='col-sm-8 col-xs-12']/div[3]//li//a")
        for link_container in dvc_container:
            link = link_container.get_attribute("href")
            link_dvc.append(link)

    file_name = file_path
    with open(file_name, "w") as file:
        for link in link_dvc:
            file.write(link + "\n")
    return link_dvc

# Xử lí Processing

In [171]:
def get_txt_from_dvc(url, file_path):
    """
    Function getting TXT files used in Model Langchain.
    
    Input: url of Public Sector
    Output: A string of processed html
            File .txt
    """
    driver = webdriver.Chrome()
    driver.get(url)
    
    # Xem chi tiết
    driver.find_element("xpath","//a[text()='Xem chi tiết']").click()
    time.sleep(2)
    
    # Lấy html
    div_element = driver.find_element("xpath","//div[@class='modal-body']")
    div_html = div_element.get_attribute('innerHTML')
    
    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(div_html, 'html.parser')
    html_tables = []
    final_str = ''
    
    link_spans = soup.find_all('span', class_='link')
    
    for span in link_spans:
          # Lấy ra mã từ thuộc tính onclick
          onclick_value = span.get('onclick')
          start_index = onclick_value.find("(") + 1
          end_index = onclick_value.find(")")
          maudon_id = onclick_value[start_index:end_index]
    
          # Nối mã vào tên file
          span.string += f'(https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma=+{maudon_id})'
    
    # Nếu phần tử này được tìm thấy
    result = ''
    duplicated = []
    link_spans = soup.find_all('span', class_='link')
    
    for span in link_spans:
          # Lấy ra mã từ thuộc tính onclick
          onclick_value = span.get('onclick')
          start_index = onclick_value.find("(") + 1
          end_index = onclick_value.find(")")
          maudon_id = onclick_value[start_index:end_index]
    
          # Nối mã vào tên file
          span.string += f'(https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma=+{maudon_id})'
    
    # Nếu phần tử này được tìm thấy
    if soup:
        added_content = []  # Danh sách để lưu trữ các nội dung đã thêm vào kết quả
        added_tables = []   # Danh sách để lưu trữ các bảng đã thêm vào kết quả
        for element in soup.descendants:
            if element is not None:
                if element.name == 'table':
                    # Kiểm tra xem bảng đã được thêm vào kết quả chưa
                    if element not in added_tables:
                        # Chuyển đổi HTML của bảng thành văn bản Markdown
                        markdown_table = table_to_markdown(element)
                        # Thêm văn bản Markdown của bảng vào kết quả
                        result += markdown_table + '\n'
                        # Thêm bảng vào danh sách đã thêm
                        added_tables.append(element)
                else:
                    if element.find_parent('table') is None:
                      if element.string and str(element.string).strip():
                        result += str(element.string).strip() + '\n'
                        # Thêm nội dung vào danh sách đã thêm
                        added_content.append(str(element.string).strip())
                          
    sentences = result.split('\n')
    unique_sentences = []
    for i, sentence in enumerate(sentences):
        if i == 0 or sentence != sentences[i-1]:
            unique_sentences.append(sentence)
    unique_result = '\n'.join(unique_sentences)
    to_remove = "Doan nay da duoc fix"

    unique_result = unique_result.replace(to_remove, "")
    
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(unique_result)
    driver.quit()
    return unique_result

In [170]:
url = "https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-nganh-doc.html?ma_thu_tuc=1.004222"
file_path = "E:\Downloads\Dki_thuongtru.txt"
x = get_txt_from_dvc(url,file_path)
print(x)